# Importing Libraries

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 100)

from sklearn.utils import shuffle

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set_style("darkgrid")

import warnings
warnings.filterwarnings("ignore")

# Data Cleaning (Preliminary)

__This section covers the preliminary data cleaning steps that are used for any future analysis path. Hence, it will focus on removing irrelevant data and deals with missing data. It will not include any drastic reformation of the original dataset.__

## Tournament Details

This section aims to prepare a dataframe consisting of important tournament details, namely tournament type, conditions and surfaces used for different tournaments in different years.

In [55]:
# Read scraped data
tour_info = pd.read_csv('~/Desktop/Projects/atptour_analysis/csv/web_scraping/tournaments_1990-2019.csv', encoding='latin-1', header = None)

# Define header
header = ['tourney_year_id', 'tourney_order', 'tourney_type', 'tourney_name',
           'tourney_id', 'tourney_slug', 'tourney_location', 'tourney_date',
           'year', 'conditions', 'surface']

tour_info.columns = header

In [56]:
# Drop irrelevant data
tour_info.drop(columns = ['tourney_year_id','tourney_order', 'tourney_name', 'tourney_id',
                     'tourney_location','tourney_date'], axis = 1, inplace = True)

In [57]:
tour_info.isna().sum()

# Be aware of missing data in tourney_type, could do further cleaning 
# if more detailed analysis were to be be conducted on different tournament

tourney_type    1192
tourney_slug       0
year               0
conditions         0
surface            0
dtype: int64

In [58]:
# Reset index and drop original index
tour_info.reset_index(inplace = True)
tour_info.drop(columns = 'index', axis = 1, inplace = True)

# Check shape
tour_info.shape

(2198, 5)

__We have successfully created a dataframe named <code>tour_info</code> with 2198 rows x 5 columns.__

## Matches

This section aims to prepare a dataframe consisting of the matches from 1991 to 2019 as required for our analysis.

In [29]:
df = pd.read_csv('~/Desktop/Projects/atptour_analysis/csv/web_scraping/match_stats_1990-2019.csv')

In [35]:
# Check for missing values
for column in df.columns:
    if df[column].isna().sum() != 0:
        print(f"{column}:{df[column].isna().sum()}")

match_duration:303
winner_slug:303
winner_serve_rating:303
winner_aces:303
winner_double_faults:303
winner_first_serves_in:303
winner_first_serves_total:303
winner_first_serve_points_won:303
winner_first_serve_points_total:303
winner_second_serve_points_won:303
winner_second_serve_points_total:303
winner_break_points_saved:303
winner_break_points_serve_total:303
winner_service_games_played:303
winner_return_rating:303
winner_first_serve_return_won:303
winner_first_serve_return_total:303
winner_second_serve_return_won:303
winner_second_serve_return_total:303
winner_break_points_converted:303
winner_break_points_return_total:303
winner_return_games_played:303
winner_service_points_won:303
winner_service_points_total:303
winner_return_points_won:303
winner_return_points_total:303
winner_total_points_won:303
winner_total_points_total:303
loser_slug:303
loser_serve_rating:303
loser_aces:303
loser_double_faults:303
loser_first_serves_in:303
loser_first_serves_total:303
loser_first_serve_poin

In [36]:
# Visually check for missing rows
df[df['match_duration'].isna() == True]

,match_id,tourney_slug,match_duration,winner_slug,winner_serve_rating,winner_aces,winner_double_faults,winner_first_serves_in,winner_first_serves_total,winner_first_serve_points_won,winner_first_serve_points_total,winner_second_serve_points_won,winner_second_serve_points_total,winner_break_points_saved,winner_break_points_serve_total,winner_service_games_played,winner_return_rating,winner_first_serve_return_won,winner_first_serve_return_total,winner_second_serve_return_won,winner_second_serve_return_total,winner_break_points_converted,winner_break_points_return_total,winner_return_games_played,winner_service_points_won,winner_service_points_total,winner_return_points_won,winner_return_points_total,winner_total_points_won,winner_total_points_total,loser_slug,loser_serve_rating,loser_aces,loser_double_faults,loser_first_serves_in,loser_first_serves_total,loser_first_serve_points_won,loser_first_serve_points_total,loser_second_serve_points_won,loser_second_serve_points_total,loser_break_points_saved,loser_break_points_serve_total,loser_service_games_played,loser_return_rating,loser_first_serve_return_won,loser_first_serve_return_total,loser_second_serve_return_won,loser_second_serve_return_total,loser_break_points_converted,loser_break_points_return_total,loser_return_games_played,loser_service_points_won,loser_service_points_total,loser_return_points_won,loser_return_points_total,loser_total_points_won,loser_total_points_total
264,1991-408-MS027-1-14-c031-s389,milan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
274,1991-408-MS025-1-4-j006-r204,milan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
306,1991-424-MS030-1-3-p012-c029,san-francisco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
339,1991-201-MS026-1-1-w038-d044,guaruja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
865,1991-329-MS043-1-19-c023-t111,tokyo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52673,2007-1720-QS026-1-4-i230-w567,bangkok,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63666,2010-468-QS008-2-7-p701-n605,estoril,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63670,2010-468-QS015-2-2-m984-bd42,estoril,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101129,2019-6932-QS015-1-2-mo44-d0ar,rio-de-janeiro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

It looks like all 303 missing rows are the same, maybe due to cancelled matches, we will remove all of them as they are not useful to the analysis and may interfere with any model we may run.

In [37]:
original_rows = df.shape[0]
print (f"Original No. Of Rows: {original_rows}")
df.dropna(inplace = True)
new_rows = df.shape[0]
print (f"New No. Of Rows: {new_rows}")
print (f"{original_rows - new_rows} number of rows were dropped !")

Original No. Of Rows: 104307
New No. Of Rows: 104004
303 number of rows were dropped !


In [38]:
# Cross-check for missing values
for column in df.columns:
    if df[column].isna().sum() != 0:
        print(f"{column}:{df[column].isna().sum()}")

No output results, confirming that there are no missing values in the dataframe ! Great !

In [41]:
# Reset Index as we dropped 303 rows
df.reset_index(inplace = True)
df.drop(columns = ['index'], axis = 1, inplace = True)

df.shape

(104004, 57)

__We have successfully created a dataframe named <code>df</code> with 104004 rows x 57 columns.__

## Map Respective Tournament Conditions & Surfaces

The section aims to map the tournament-specific conditions and surface from the <code>tour_info</code> dataframe onto the main <code>df</code> dataframe.

In [59]:
tour_info.head()

,tourney_type,tourney_slug,year,conditions,surface
0,NaN,adelaide,1990,Outdoor,Hard
1,NaN,wellington,1990,Outdoor,Hard
2,NaN,sydney,1990,Outdoor,Hard
3,NaN,auckland,1990,Outdoor,Hard
4,Grand Slam,australian-open,1990,Outdoor,Hard


In [60]:
df.head()

,match_id,year,tourney_slug,conditions,surface,match_duration,winner_slug,winner_serve_rating,winner_aces,winner_double_faults,winner_first_serves_in,winner_first_serves_total,winner_first_serve_points_won,winner_first_serve_points_total,winner_second_serve_points_won,winner_second_serve_points_total,winner_break_points_saved,winner_break_points_serve_total,winner_service_games_played,winner_return_rating,winner_first_serve_return_won,winner_first_serve_return_total,winner_second_serve_return_won,winner_second_serve_return_total,winner_break_points_converted,winner_break_points_return_total,winner_return_games_played,winner_service_points_won,winner_service_points_total,winner_return_points_won,winner_return_points_total,winner_total_points_won,winner_total_points_total,loser_slug,loser_serve_rating,loser_aces,loser_double_faults,loser_first_serves_in,loser_first_serves_total,loser_first_serve_points_won,loser_first_serve_points_total,loser_second_serve_points_won,loser_second_serve_points_total,loser_break_points_saved,loser_break_points_serve_total,loser_service_games_played,loser_return_rating,loser_first_serve_return_won,loser_first_serve_return_total,loser_second_serve_return_won,loser_second_serve_return_total,loser_break_points_converted,loser_break_points_return_total,loser_return_games_played,loser_service_points_won,loser_service_points_total,loser_return_points_won,loser_return_points_total,loser_total_points_won,loser_total_points_total
0,1991-7308-MS001-5-1-k181-s351,1991.0,adelaide,NaN,NaN,89.0,k181,269.0,1.0,2.0,44.0,65.0,31.0,44.0,12.0,21.0,2.0,5.0,12.0,182.0,16.0,47.0,21.0,36.0,4.0,7.0,12.0,43.0,65.0,37.0,83.0,80.0,148.0,s351,238.0,12.0,5.0,47.0,83.0,31.0,47.0,15.0,36.0,3.0,7.0,12.0,158.0,13.0,44.0,9.0,21.0,3.0,5.0,12.0,46.0,83.0,22.0,65.0,68.0,148.0
1,1991-7308-MS003-4-2-s351-c243,1991.0,adelaide,NaN,NaN,104.0,s351,272.0,7.0,4.0,37.0,80.0,26.0,37.0,27.0,43.0,5.0,6.0,11.0,155.0,11.0,40.0,13.0,31.0,2.0,3.0,11.0,53.0,80.0,24.0,71.0,77.0,151.0,c243,268.0,2.0,2.0,40.0,71.0,29.0,40.0,18.0,31.0,1.0,3.0,11.0,93.0,11.0,37.0,16.0,43.0,1.0,6.0,11.0,47.0,71.0,27.0,80.0,74.0,151.0
2,1991-7308-MS002-4-1-k181-l206,1991.0,adelaide,NaN,NaN,80.0,k181,281.0,3.0,0.0,0.0,65.0,29.0,46.0,12.0,19.0,0.0,2.0,11.0,187.0,12.0,36.0,20.0,33.0,4.0,7.0,11.0,41.0,65.0,32.0,69.0,73.0,134.0,l206,222.0,1.0,0.0,36.0,69.0,24.0,36.0,13.0,33.0,3.0,7.0,11.0,192.0,17.0,46.0,7.0,19.0,2.0,2.0,11.0,37.0,69.0,24.0,65.0,61.0,134.0
3,1991-7308-MS007-3-4-c243-s367,1991.0,adelaide,NaN,NaN,80.0,c243,306.0,5.0,2.0,41.0,61.0,31.0,41.0,14.0,20.0,0.0,1.0,11.0,161.0,6.0,34.0,15.0,27.0,2.0,3.0,10.0,45.0,61.0,21.0,61.0,66.0,122.0,s367,258.0,1.0,5.0,34.0,61.0,28.0,34.0,12.0,27.0,1.0,3.0,10.0,163.0,10.0,41.0,6.0,20.0,1.0,1.0,11.0,40.0,61.0,16.0,61.0,56.0,122.0
4,1991-7308-MS006-3-3-s351-a031,1991.0,adelaide,NaN,NaN,90.0,s351,287.0,5.0,2.0,45.0,74.0,34.0,45.0,18.0,29.0,2.0,4.0,14.0,148.0,18.0,57.0,12.0,24.0,3.0,7.0,13.0,52.0,74.0,30.0,81.0,82.0,155.0,a031,264.0,2.0,2.0,57.0,81.0,39.0,57.0,12.0,24.0,4.0,7.0,13.0,126.0,11.0,45.0,11.0,29.0,2.0,4.0,14.0,51.0,81.0,22.0,74.0,73.0,155.0


In [61]:
# Create 2 empty columns before mapping values from tournament data
nan_value = float("NaN")
df['year'] = nan_value
df["conditions"] = nan_value
df["surface"] = nan_value

In [62]:
rearranged_list = ['match_id', 'year', 'tourney_slug', 'conditions', 'surface', 'match_duration',
                'winner_slug', 'winner_serve_rating', 'winner_aces', 'winner_double_faults', 
               'winner_first_serves_in', 'winner_first_serves_total', 'winner_first_serve_points_won',
                'winner_first_serve_points_total', 'winner_second_serve_points_won',
                'winner_second_serve_points_total', 'winner_break_points_saved', 'winner_break_points_serve_total',
                'winner_service_games_played', 'winner_return_rating', 'winner_first_serve_return_won',
                'winner_first_serve_return_total', 'winner_second_serve_return_won', 'winner_second_serve_return_total',
                'winner_break_points_converted', 'winner_break_points_return_total', 'winner_return_games_played',
                'winner_service_points_won', 'winner_service_points_total', 'winner_return_points_won',
                'winner_return_points_total', 'winner_total_points_won', 'winner_total_points_total',
                'loser_slug', 'loser_serve_rating', 'loser_aces', 'loser_double_faults', 'loser_first_serves_in',
                'loser_first_serves_total', 'loser_first_serve_points_won', 'loser_first_serve_points_total',
                'loser_second_serve_points_won', 'loser_second_serve_points_total', 'loser_break_points_saved',
                'loser_break_points_serve_total', 'loser_service_games_played', 'loser_return_rating',
                'loser_first_serve_return_won', 'loser_first_serve_return_total', 'loser_second_serve_return_won',
                'loser_second_serve_return_total', 'loser_break_points_converted', 'loser_break_points_return_total',
                'loser_return_games_played', 'loser_service_points_won', 'loser_service_points_total',
                'loser_return_points_won', 'loser_return_points_total', 'loser_total_points_won', 'loser_total_points_total']

# Rearrange the column sequence so that all the statistics are placed at the back
df = df.reindex(columns = rearranged_list)
# ** This step is not required but this structure is more organized **

In [63]:
# Populate the year column so that it is easily comparable to the tournament dataframe
for i in range(0, len(df)):
    df['year'][i] = round(int(df['match_id'][i][0:4]),4)

In [67]:
# Populate the df columns for conditions and surface based on name and year of tournament
for i in range(0, len(df)):
    tourney_filter = tour_info[tour_info['tourney_slug'] == df['tourney_slug'][i]]
    tourney_filter2 = tourney_filter[tourney_filter['year'] == df['year'][i]]
    df['conditions'][i] = tourney_filter2['conditions'].values[0]
    df['surface'][i] = tourney_filter2['surface'].values[0]
    
## ** This code takes 6 minutes **

In [68]:
df.shape

(104004, 60)

__We have successfully created a dataframe named <code>df</code> with 104004 rows x 60 columns.__

## Check For Rows with Unusual Data

In [69]:
original_row = df.shape[0]

In [70]:
df.describe()

,year,match_duration,winner_serve_rating,winner_aces,winner_double_faults,winner_first_serves_in,winner_first_serves_total,winner_first_serve_points_won,winner_first_serve_points_total,winner_second_serve_points_won,winner_second_serve_points_total,winner_break_points_saved,winner_break_points_serve_total,winner_service_games_played,winner_return_rating,winner_first_serve_return_won,winner_first_serve_return_total,winner_second_serve_return_won,winner_second_serve_return_total,winner_break_points_converted,winner_break_points_return_total,winner_return_games_played,winner_service_points_won,winner_service_points_total,winner_return_points_won,winner_return_points_total,winner_total_points_won,winner_total_points_total,loser_serve_rating,loser_aces,loser_double_faults,loser_first_serves_in,loser_first_serves_total,loser_first_serve_points_won,loser_first_serve_points_total,loser_second_serve_points_won,loser_second_serve_points_total,loser_break_points_saved,loser_break_points_serve_total,loser_service_games_played,loser_return_rating,loser_first_serve_return_won,loser_first_serve_return_total,loser_second_serve_return_won,loser_second_serve_return_total,loser_break_points_converted,loser_break_points_return_total,loser_return_games_played,loser_service_points_won,loser_service_points_total,loser_return_points_won,loser_return_points_total,loser_total_points_won,loser_total_points_total
count,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000,104004.000000
mean,2005.953223,99.218819,279.116861,6.185108,2.681589,44.754856,75.659484,34.678051,46.083362,16.250663,29.578045,3.310652,4.908609,12.001990,173.547642,15.753865,46.458934,17.442108,31.964088,3.877284,8.569853,11.822718,50.926339,75.657917,33.194964,78.420638,84.119640,154.075584,232.256404,4.587458,3.456559,46.484203,78.463578,30.744173,46.486193,14.523595,31.977943,4.694012,8.568834,11.822526,109.952406,11.433483,46.039354,13.324670,29.558921,1.599429,4.908754,12.001769,45.263461,78.455896,24.749692,75.574824,69.984078,153.971011
std,8.436766,39.063761,37.165007,5.131846,2.334390,20.428946,29.219977,13.596259,18.906448,6.942467,13.227064,3.093678,4.086679,4.081389,45.080593,7.047398,19.074109,7.387309,13.229004,1.663391,4.077878,4.090687,18.307890,29.222705,11.570829,29.152172,27.545956,56.819662,40.733793,4.482772,2.589302,19.043324,29.091496,14.173830,19.043478,7.150955,13.210576,3.237842,4.078726,4.090822,45.012863,7.057741,18.969632,7.422741,13.248286,1.545545,4.086533,4.081569,19.533207,29.106000,12.524394,29.351944,30.053726,57.012308
min,1991.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1998.000000,72.000000,263.000000,3.000000,1.000000,32.000000,55.000000,26.000000,33.000000,12.000000,20.000000,1.000000,2.000000,9.000000,146.000000,11.000000,33.000000,13.000000,23.000000,3.000000,6.000000,9.000000,38.000000,55.000000,26.000000,58.000000,65.000000,

### Match Duration & Total Points Played

With a quick research, we can find out that the shortest completed match was completed in 28 minutes. The match was Jarkko Nieminen d. Bernard Tomic 6-0 6-1 in the first round of Miami Open 2014 edition. As the dataset consists of interrupted or retired matches, we can safely assume any matches with duration below 28 mins were prematurely ended.

In [71]:
for column in df.columns[5:]:
    print(f"{column}:{df[column].value_counts()}")

match_duration:70.0      1459
72.0      1409
75.0      1405
76.0      1403
80.0      1394
          ... 
341.0        1
1147.0       1
304.0        1
314.0        1
326.0        1
Name: match_duration, Length: 322, dtype: int64
winner_slug:f324    1169
n409     946
d643     854
f401     696
mc10     642
        ... 
bk41       1
k858       1
c663       1
cb04       1
j400       1
Name: winner_slug, Length: 2096, dtype: int64
winner_serve_rating:288.0    1565
279.0    1562
282.0    1542
281.0    1541
285.0    1536
         ... 
385.0       1
116.0       1
167.0       1
153.0       1
175.0       1
Name: winner_serve_rating, Length: 246, dtype: int64
winner_aces:3.0      11086
2.0      10590
4.0      10462
5.0       9737
1.0       8892
6.0       8358
7.0       7079
0.0       6015
8.0       5940
9.0       4888
10.0      3950
11.0      3309
12.0      2750
13.0      2088
14.0      1744
15.0      1328
16.0      1084
17.0       890
18.0       704
19.0       603
20.0       484
21.0       391
22

loser_break_points_serve_total:7.0     11495
8.0     11083
6.0     10852
9.0      9853
5.0      8952
10.0     8685
11.0     6877
4.0      6517
12.0     5640
13.0     4374
3.0      4110
14.0     3241
15.0     2337
2.0      1991
16.0     1823
17.0     1259
0.0      1118
18.0      924
1.0       724
19.0      632
20.0      468
21.0      299
22.0      239
23.0      176
24.0      101
25.0       67
26.0       50
27.0       38
29.0       27
28.0       19
31.0       12
30.0       11
32.0        3
34.0        3
33.0        2
46.0        1
35.0        1
Name: loser_break_points_serve_total, dtype: int64
loser_service_games_played:9.0     15718
10.0    14689
8.0     11106
14.0     8705
11.0     8615
15.0     8100
13.0     6723
12.0     6698
16.0     5805
7.0      4229
17.0     3295
18.0     1667
19.0     1051
6.0      1024
20.0      884
0.0       870
21.0      797
22.0      619
23.0      568
24.0      518
25.0      486
5.0       360
26.0      326
4.0       265
3.0       203
27.0      188
28.0     

In [72]:
filter1 = df[df['match_duration'] == 28]
filter2 = filter1[filter1['year'] == 2014]
filter3 = filter2[filter2['tourney_slug'] == 'miami']
filter3

,match_id,year,tourney_slug,conditions,surface,match_duration,winner_slug,winner_serve_rating,winner_aces,winner_double_faults,winner_first_serves_in,winner_first_serves_total,winner_first_serve_points_won,winner_first_serve_points_total,winner_second_serve_points_won,winner_second_serve_points_total,winner_break_points_saved,winner_break_points_serve_total,winner_service_games_played,winner_return_rating,winner_first_serve_return_won,winner_first_serve_return_total,winner_second_serve_return_won,winner_second_serve_return_total,winner_break_points_converted,winner_break_points_return_total,winner_return_games_played,winner_service_points_won,winner_service_points_total,winner_return_points_won,winner_return_points_total,winner_total_points_won,winner_total_points_total,loser_slug,loser_serve_rating,loser_aces,loser_double_faults,loser_first_serves_in,loser_first_serves_total,loser_first_serve_points_won,loser_first_serve_points_total,loser_second_serve_points_won,loser_second_serve_points_total,loser_break_points_saved,loser_break_points_serve_total,loser_service_games_played,loser_return_rating,loser_first_serve_return_won,loser_first_serve_return_total,loser_second_serve_return_won,loser_second_serve_return_total,loser_break_points_converted,loser_break_points_return_total,loser_return_games_played,loser_service_points_won,loser_service_points_total,loser_return_points_won,loser_return_points_total,loser_total_points_won,loser_total_points_total
80739,2014-403-MS085-3-11-n289-ta46,2014.0,miami,Outdoor,Hard,28.0,n289,337.0,2.0,0.0,28.0,34.0,24.0,28.0,4.0,6.0,0.0,0.0,7.0,338.0,11.0,17.0,9.0,10.0,5.0,5.0,6.0,28.0,34.0,20.0,27.0,48.0,61.0,ta46,125.0,2.0,1.0,17.0,27.0,6.0,17.0,1.0,10.0,0.0,5.0,6.0,47.0,4.0,28.0,2.0,6.0,0.0,0.0,7.0,7.0,27.0,6.0,34.0,13.0,61.0


After a quick search of the shortest ATP match ever recorded, we will proceed to filter out interrupted matches based on several facts:
1. This match ended at 28 minutes, hence we will remove any match with a duration of 28 minutes or lower.
2. The least possible points played in a complete match is 48 points, this assumes the winner wins every single point in a BO3 format, the shortest match as displayed above played a total of 61 points, we will remove any game with 61 points or lower.

In [73]:
## Removing matches with 28 minutes or less
# Inspecting prematurely ended matches
print(f"Number of rows with match duration less than 28 minutes: {len(df[df['match_duration'] < 28])}")
      
# Removing these rows
df = df[df['match_duration'] > 28]

# Check figures again
print(f"Number of rows with match duration less than 28 minutes: {len(df[df['match_duration'] < 28])}")

Number of rows with match duration less than 28 minutes: 1770
Number of rows with match duration less than 28 minutes: 0


In [74]:
## Removing matches with 61 total points or less
# Inspecting prematurely ended matches
print(f"Number of rows with total points played less than 61 points: {len(df[df['loser_total_points_total'] < 61])}")
      
# Removing these rows
df = df[df['loser_total_points_total'] > 61]
      
# Check figures again
print(f"Number of rows with total points played less than 61 points: {len(df[df['loser_total_points_total'] < 61])}")

Number of rows with total points played less than 61 points: 1352
Number of rows with total points played less than 61 points: 0


In this section, we removed a total of __3122 rows__ of interrupted matches.

### Serve & Return Ratings

In this section, we wish to remove undefined serve & return ratings present in the dataset.

In [75]:
# Inspecting missing winner_serve_rating values
print(f"Number of rows missing winner_serve_rating values: {str(len(df[df['winner_serve_rating'] == 0]))}") # 800 rows

# Removing these rows
df = df[df['winner_serve_rating'] != 0]
      
# Check missing values again
print(f"Number of rows missing winner_serve_rating values: {str(len(df[df['winner_serve_rating'] == 0]))}") # 0 rows
      


# Inspecting missing loser_serve_rating values
print(f"Number of rows missing loser_serve_rating values: {str(len(df[df['loser_serve_rating'] == 0]))}")   # 7 rows
      
# Removing these rows
df = df[df['loser_serve_rating'] != 0]

# Check missing values again      
print(f"Number of rows missing loser_serve_rating values: {str(len(df[df['loser_serve_rating'] == 0]))}") # 0 rows

Number of rows missing winner_serve_rating values: 1
Number of rows missing winner_serve_rating values: 0
Number of rows missing loser_serve_rating values: 0
Number of rows missing loser_serve_rating values: 0


In [76]:
# Inspecting missing winner_return_rating values
print(f"Number of rows missing winner_return_rating values: {str(len(df[df['winner_return_rating'] == 0]))}") # 5 rows

# Removing these rows
df = df[df['winner_return_rating'] != 0]
      
# Check missing values again
print(f"Number of rows missing winner_return_rating values: {str(len(df[df['winner_return_rating'] == 0]))}") # 0 rows
      


# Inspecting missing loser_return_rating values
print(f"Number of rows missing loser_return_rating values: {str(len(df[df['loser_return_rating'] == 0]))}")   # 49 rows
      
# Removing these rows
df = df[df['loser_return_rating'] != 0]

# Check missing values again      
print(f"Number of rows missing loser_return_rating values: {str(len(df[df['loser_return_rating'] == 0]))}") # 0 rows

Number of rows missing winner_return_rating values: 1
Number of rows missing winner_return_rating values: 0
Number of rows missing loser_return_rating values: 6
Number of rows missing loser_return_rating values: 0


__Section Summary: 8 rows__ were removed from this section due to undefined serve ratings.

### First Serves

In this section, we aim to remove missing data from First Serves Statistics, including <code>first_serves_in</code> & <code>first_serves_total</code>. We will remove all rows with 0 values as it is unreasonable for a player to land 0 first serves or serve 0 first serves in total in a completed match. Therefore, we will remove these rows.

In [77]:
# Inspecting missing first_serves_in values
print(f"Number of rows missing winner_first_serves_in values: {str(len(df[df['winner_first_serves_in'] == 0]))}") # 3507 rows

# # Removing these rows
df = df[df['winner_first_serves_in'] != 0]
      
# # Check missing values again
print(f"Number of rows missing winner_first_serves_in values: {str(len(df[df['winner_first_serves_in'] == 0]))}") # 0 rows
      

      
# Inspecting missing first_serves_in values
print(f"Number of rows missing loser_first_serves_in values: {str(len(df[df['loser_first_serves_in'] == 0]))}") # 2 rows

# # Removing these rows
df = df[df['loser_first_serves_in'] != 0]
      
# # Check missing values again
print(f"Number of rows missing loser_first_serves_in values: {str(len(df[df['loser_first_serves_in'] == 0]))}") # 0 rows

Number of rows missing winner_first_serves_in values: 3319
Number of rows missing winner_first_serves_in values: 0
Number of rows missing loser_first_serves_in values: 0
Number of rows missing loser_first_serves_in values: 0


In [78]:
# Inspecting missing first_serves_total values
print(f"Number of rows missing winner_first_serves_total values: {str(len(df[df['winner_first_serves_total'] == 0]))}") # 0 rows

# # Removing these rows
df = df[df['winner_first_serves_total'] != 0]
      
# # Check missing values again
print(f"Number of rows missing winner_first_serves_total values: {str(len(df[df['winner_first_serves_total'] == 0]))}") # 0 rows
      
      
    
# Inspecting missing first_serves_total values
print(f"Number of rows missing loser_first_serves_total values: {str(len(df[df['loser_first_serves_total'] == 0]))}") # 0 rows

# # Removing these rows
df = df[df['loser_first_serves_total'] != 0]
      
# # Check missing values again
print(f"Number of rows missing loser_first_serves_total values: {str(len(df[df['loser_first_serves_total'] == 0]))}") # 0 rows

Number of rows missing winner_first_serves_total values: 0
Number of rows missing winner_first_serves_total values: 0
Number of rows missing loser_first_serves_total values: 0
Number of rows missing loser_first_serves_total values: 0


__Section Summary: 3319 rows__ were removed from this section due to invalid first serves data.

### Points & Games Requirement 

In this section, we aim to remove more invalid data based on the simple scoring system from the ATP. At a minimum, a player would have to play at least 24 service points and 24 return points; a player would have to play at least 6 service games and 6 returns games. As a requirement to win a match in 2 sets, the winner is also required to win at least 48 points in a completed match.

In [79]:
original = df.shape[0]  # 97488 rows

In [80]:
# Removing games where players played less than 6 service games in total, indicating incomplete matches
df = df[df['winner_service_games_played'] >= 6]
df = df[df['loser_service_games_played'] >= 6]

# Removing games where players played less than 6 return games in total, indicating incomplete matches
df = df[df['winner_return_games_played'] >= 6]
df = df[df['loser_return_games_played'] >= 6]

In [81]:
# Removing all games with players playing less than 24 points on serve
df = df[df['winner_service_points_total'] > 24]
df = df[df['loser_service_points_total'] > 24]

# Removing all games with players playing less than 24 points on return
df = df[df['winner_return_points_total'] > 24]
df = df[df['loser_return_points_total'] > 24]

In [82]:
# Remove any match with winner winning less than 48 points
df = df[df['winner_total_points_won'] >= 48]

In [83]:
new = df.shape[0]
print (f"We have successfully removed {original - new} number of rows due to incomplete matches")

We have successfully removed 637 number of rows due to incomplete matches


### Remaining Incorrect Data

In [84]:
for column in df.columns[10:]:
    print(f"{column}: {df[column].min()}")

winner_first_serves_in: 9.0
winner_first_serves_total: 25.0
winner_first_serve_points_won: 8.0
winner_first_serve_points_total: 9.0
winner_second_serve_points_won: 0.0
winner_second_serve_points_total: 0.0
winner_break_points_saved: 0.0
winner_break_points_serve_total: 0.0
winner_service_games_played: 6.0
winner_return_rating: 19.0
winner_first_serve_return_won: 0.0
winner_first_serve_return_total: 7.0
winner_second_serve_return_won: 0.0
winner_second_serve_return_total: 0.0
winner_break_points_converted: 0.0
winner_break_points_return_total: 0.0
winner_return_games_played: 6.0
winner_service_points_won: 18.0
winner_service_points_total: 25.0
winner_return_points_won: 6.0
winner_return_points_total: 27.0
winner_total_points_won: 48.0
winner_total_points_total: 62.0
loser_slug: a005
loser_serve_rating: 63.0
loser_aces: 0.0
loser_double_faults: 0.0
loser_first_serves_in: 7.0
loser_first_serves_total: 27.0
loser_first_serve_points_won: 1.0
loser_first_serve_points_total: 7.0
loser_second_

In [85]:
# Remove match where loser_break_points_saved was recorded as -6
df = df[df['loser_break_points_saved'] > 0]

### Short Summary of Section

In [86]:
# Reset Index
df.reset_index(inplace = True)
df.drop(columns = 'index', inplace = True)

In [87]:
new_row = df.shape[0]
print(f"In this section, we have removed a total of {original_row - new_row} rows. The clean dataset now contains {df.shape[0]} rows x {df.shape[1]} columns.")

In this section, we have removed a total of 11845 rows. The clean dataset now contains 92159 rows x 60 columns.


In [88]:
# Save clean dataframe for easy accessibility for future use
df.to_csv('clean_df_v1.0.csv', index = False)

__The prelimiary stage of data cleaning is completed. We have successfully created a dataframe with 92159 rows x 60 columns. The dataframe was saved as a csv file named <code>clean_df_v1.0</code>__

# Data Cleaning (Match Statistics Analysis)

This section will continue the data cleaning process specific for match statistics analysis, it will mainly include feature engineering work, the final dataset produced by this section will be suitable for simple statistical analysis.

In [16]:
df = pd.read_csv('clean_df_v1.0.csv')

## Plans on Feature Engineering

An example of a match statistics from official ATP website is shown below for references:

<img src = 'stats_example.png'>

As the web scrapping process only extract raw values, the percentage points included in the image above are not included in the dataset. So let's begin feature engineering !

By looking at the columns, we can feature engineer the following percentage metrics:
1. First Serves In %
2. First Serves Points Won %
3. Second Serves Points Won %
4. Break Point Save % (To evaluate serve performance during pressure point) * When feature engineering this metric,remember to check total break point faced value, e.g. if player did not face break point *
5. First Serves Return Points Won %
6. Second Serves Return Points Won %
7. Break Point Conversion % (To evaluate return performance during pressure point) * Feature engineering include evaluating opponents' break point values
8. Service Points Won %
9. Return Points Won %
10. Total Points Won %

Additional Metrics:
11. Aces:Double Faults Ratio
12. Aces Per Minute, to evaluate comparable ace performance with regards to time as a longer match would result in more aces
13. Difference In First Serves In % (If a player get more first serves in, more points will be played on his first serve, which is normally his stronger serve as he has his second serve to spare, this usually gives the server an added advantage to win the point. This is evident as first serve points won % are usually significantly higher than second serve points won %)
14. Serve Rating:Return Rating Ratio (To evaluate which part of the game is more crucial for a win)
15. Serve:Return Points Won Ratio (Another metric to measure the importance of service vs return games)
16. Difference in Total Return Points Played (More return points played indicates the opponent's serve is always under pressure, hence the winner is expected to have more return points played)

## Percentage Metrics

### First Serves In %

__First Serves In %__ is derived by <code>first_serves_in</code> / <code>first_serves_total</code>

In [17]:
winner_first_serves_in_perc = []
loser_first_serves_in_perc = []

# Calculate % score
for i in range(0,len(df)):
    winner_first_serves_in_perc.append(df['winner_first_serves_in'][i]/df['winner_first_serves_total'][i])
    loser_first_serves_in_perc.append(df['loser_first_serves_in'][i]/df['loser_first_serves_total'][i])

# Create new columns with % score
df['winner_first_serves_in_%'] = winner_first_serves_in_perc
df['loser_first_serves_in_%'] = loser_first_serves_in_perc

### First Serves Points Won %

__First Serves Points Won %__ is derived by <code>first_serve_points_won</code> / <code>first_serve_points_total</code>

In [18]:
winner_first_serves_won_perc = []
loser_first_serves_won_perc = []

# Calculate % score
for i in range(0,len(df)):
    winner_first_serves_won_perc.append(df['winner_first_serve_points_won'][i]/df['winner_first_serve_points_total'][i])
    loser_first_serves_won_perc.append(df['loser_first_serve_points_won'][i]/df['loser_first_serve_points_total'][i])

# Create new columns with % score
df['winner_first_serves_points_won_%'] = winner_first_serves_won_perc
df['loser_first_serves_points_won_%'] = loser_first_serves_won_perc

### Second Serves Points Won %

__Second Serves Points Won %__ is derived by <code>second_serve_points_won</code> / <code>second_serve_points_total</code>

In [19]:
winner_second_serves_won_perc = []
loser_second_serves_won_perc = []

# Calculate % score
for i in range(0,len(df)):
    winner_second_serves_won_perc.append(df['winner_second_serve_points_won'][i]/df['winner_second_serve_points_total'][i])
    loser_second_serves_won_perc.append(df['loser_second_serve_points_won'][i]/df['loser_second_serve_points_total'][i])

# Create new columns with % score
df['winner_second_serves_points_won_%'] = winner_second_serves_won_perc
df['loser_second_serves_points_won_%'] = loser_second_serves_won_perc

In [20]:
for column in df.columns[-2:]:
    print(f"{column}: {df[column].isnull().sum()}")

winner_second_serves_points_won_%: 3
loser_second_serves_points_won_%: 3


6 rows containing null values are identified at this stage, let's visually inspect what went wrong.

In [21]:
df[df['winner_second_serves_points_won_%'].isna() == True]

,match_id,year,tourney_slug,conditions,surface,match_duration,winner_slug,winner_serve_rating,winner_aces,winner_double_faults,winner_first_serves_in,winner_first_serves_total,winner_first_serve_points_won,winner_first_serve_points_total,winner_second_serve_points_won,winner_second_serve_points_total,winner_break_points_saved,winner_break_points_serve_total,winner_service_games_played,winner_return_rating,winner_first_serve_return_won,winner_first_serve_return_total,winner_second_serve_return_won,winner_second_serve_return_total,winner_break_points_converted,winner_break_points_return_total,winner_return_games_played,winner_service_points_won,winner_service_points_total,winner_return_points_won,winner_return_points_total,winner_total_points_won,winner_total_points_total,loser_slug,loser_serve_rating,loser_aces,loser_double_faults,loser_first_serves_in,loser_first_serves_total,loser_first_serve_points_won,loser_first_serve_points_total,loser_second_serve_points_won,loser_second_serve_points_total,loser_break_points_saved,loser_break_points_serve_total,loser_service_games_played,loser_return_rating,loser_first_serve_return_won,loser_first_serve_return_total,loser_second_serve_return_won,loser_second_serve_return_total,loser_break_points_converted,loser_break_points_return_total,loser_return_games_played,loser_service_points_won,loser_service_points_total,loser_return_points_won,loser_return_points_total,loser_total_points_won,loser_total_points_total,winner_first_serves_in_%,loser_first_serves_in_%,winner_first_serves_points_won_%,loser_first_serves_points_won_%,winner_second_serves_points_won_%,loser_second_serves_points_won_%
33455,2003-325-MS019-1-10-m824-r383,2003.0,palermo,Outdoor,Clay,76.0,m824,261.0,1.0,0.0,50.0,50.0,36.0,50.0,0.0,0.0,1.0,2.0,9.0,208.0,18.0,45.0,4.0,6.0,4.0,7.0,9.0,36.0,50.0,22.0,51.0,58.0,101.0,r383,236.0,0.0,0.0,45.0,51.0,27.0,45.0,2.0,6.0,3.0,7.0,9.0,89.0,14.0,50.0,0.0,0.0,1.0,2.0,9.0,29.0,51.0,14.0,50.0,43.0,101.0,1.0,0.882353,0.720000,0.600000,NaN,0.333333
43117,2007-416-QS013-2-4-m655-k336,2007.0,rome,Outdoor,Clay,97.0,m655,164.0,0.0,0.0,102.0,102.0,38.0,102.0,0.0,0.0,6.0,19.0,18.0,221.0,69.0,111.0,0.0,0.0,13.0,15.0,18.0,38.0,102.0,69.0,111.0,107.0,213.0,k336,165.0,0.0,0.0,111.0,111.0,42.0,111.0,0.0,0.0,2.0,15.0,18.0,203.0,64.0,102.0,0.0,0.0,13.0,19.0,18.0,42.0,111.0,64.0,102.0,106.0,213.0,1.0,1.000000,0.372549,0.378378,NaN,NaN
49520,2009-580-MS115-4-38-ma30-l397,2009.0,australian-open,Outdoor,Hard,262.0,ma30,317.0,47.0,0.0,194.0,194.0,148.0,194.0,0.0,0.0,4.0,6.0,36.0,64.0,51.0,196.0,0.0,0.0,2.0,6.0,35.0,148.0,194.0,51.0,196.0,199.0,390.0,l397,268.0,0.0,0.0,196.0,196.0,145.0,196.0,0.0,0.0,4.0,6.0,35.0,62.0,46.0,194.0,0.0,0.0,2.0,6.0,36.0,145.0,196.0,46.0,194.0,191.0,390.0,1.0,1.000000,0.762887,0.739796,NaN,NaN


As we can see from the dataframe, as the second serves points won % was calculated with the formula 

<code>second_serve_points_won</code> / <code>second_serve_points_total</code>

Null values are calculated when second_serve_points_total scores are recorded as 0, as we can not have 0 as our denominator, to fix this problem, we wil convert all such null values to 0.

In [22]:
df['winner_second_serves_points_won_%'].fillna(value = 0, inplace = True)
df['loser_second_serves_points_won_%'].fillna(value = 0, inplace = True)

In [23]:
for column in df.columns[-2:]:
    print(f"{column}: {df[column].isnull().sum()}")

winner_second_serves_points_won_%: 0
loser_second_serves_points_won_%: 0


### Break Point Save %

__Break Point Save %__ is derived by <code>break_points_saved</code> / <code>break_points_serve_total</code>

* **Be careful when dealing with players who faced 0 break points during the match as the formula would result in 0.**

To solve this problem, we will first define another column with boolean values indicating if the player had faced a break point during the match.

In [24]:
# Create a column indicating if player faced any break points during the match
df['winner_faced_break_points?'] = 0
df['loser_faced_break_points?'] = 0

for i in range(0, len(df)):
    if df['winner_break_points_serve_total'][i] != 0:
        df['winner_faced_break_points?'][i] = 1

for i in range(0, len(df)):
    if df['loser_break_points_serve_total'][i] != 0:
        df['loser_faced_break_points?'][i] = 1

In [25]:
winner_break_point_save_perc = []
loser_break_point_save_perc = []

# Calculate % score
for i in range(0,len(df)):
    winner_break_point_save_perc.append(df['winner_break_points_saved'][i]/df['winner_break_points_serve_total'][i])
    loser_break_point_save_perc.append(df['loser_break_points_saved'][i]/df['loser_break_points_serve_total'][i])

# Create new columns with % score
df['winner_break_point_save_%'] = winner_break_point_save_perc
df['loser_break_point_save_%'] = loser_break_point_save_perc

In [26]:
for column in df.columns[-2:]:
    print(f"{column}: {df[column].isnull().sum()}")

winner_break_point_save_%: 10638
loser_break_point_save_%: 0


We noticed 10638 rows with null values on the winner stats, but 0 rows from the loser stats. This is expected as it is very possible for the winner to win the match without facing a single break point, hence we have 0 as the denominator in the equation to obtain break point save %. We will once again convert all of them to 0, but this will not affect our analysis as we created a boolean column, indicating if a player faced a break point during the match, therefore retaining the information that we had from the original dataset.

In [27]:
df['winner_break_point_save_%'].fillna(value = 0, inplace = True)

In [28]:
for column in df.columns[-2:]:
    print(f"{column}: {df[column].isnull().sum()}")

winner_break_point_save_%: 0
loser_break_point_save_%: 0


### First Serves Return Point Won %

__First Serves Return Points Won %__ is derived by <code>first_serve_return_won</code> / <code>first_serve_return_total</code>

In [29]:
winner_first_serves_return_won_perc = []
loser_first_serves_return_won_perc = []

# Calculate % score
for i in range(0,len(df)):
    winner_first_serves_return_won_perc.append(df['winner_first_serve_return_won'][i]/df['winner_first_serve_return_total'][i])
    loser_first_serves_return_won_perc.append(df['loser_first_serve_return_won'][i]/df['loser_first_serve_return_total'][i])

# Create new columns with % score
df['winner_first_serves_return_points_won_%'] = winner_first_serves_return_won_perc
df['loser_first_serves_return_points_won_%'] = loser_first_serves_return_won_perc

### Second Serves Return Points Won %

__Second Serves Return Points Won %__ is derived by <code>second_serve_return_won</code> / <code>second_serve_return_total</code>

In [30]:
winner_second_serves_return_won_perc = []
loser_second_serves_return_won_perc = []

# Calculate % score
for i in range(0,len(df)):
    winner_second_serves_return_won_perc.append(df['winner_second_serve_return_won'][i]/df['winner_second_serve_return_total'][i])
    loser_second_serves_return_won_perc.append(df['loser_second_serve_return_won'][i]/df['loser_second_serve_return_total'][i])

# Create new columns with % score
df['winner_second_serves_return_points_won_%'] = winner_second_serves_return_won_perc
df['loser_second_serves_return_points_won_%'] = loser_second_serves_return_won_perc

In [31]:
for column in df.columns[-2:]:
    print(f"{column}: {df[column].isnull().sum()}")

winner_second_serves_return_points_won_%: 3
loser_second_serves_return_points_won_%: 3


Similarly, we have the 0 denominator problem, we will fill null values with 0.

In [32]:
df['winner_second_serves_return_points_won_%'].fillna(value = 0, inplace = True)
df['loser_second_serves_return_points_won_%'].fillna(value = 0, inplace = True)

In [33]:
for column in df.columns[-2:]:
    print(f"{column}: {df[column].isnull().sum()}")

winner_second_serves_return_points_won_%: 0
loser_second_serves_return_points_won_%: 0


### Break Point Conversion %

__Break Point Conversion %__ is derived by <code>break_points_converted</code> / <code>break_points_return_total</code>

Similar to Break Point Save %, we will create separate boolean-valued column, indicating if the player had any break point opportunities during the match.

In [34]:
# Create a column indicating if player faced any break points during the match
df['winner_any_break_points_chance?'] = 0
df['loser_any_break_points_chance?'] = 0

for i in range(0, len(df)):
    if df['winner_break_points_return_total'][i] != 0:
        df['winner_any_break_points_chance?'][i] = 1

for i in range(0, len(df)):
    if df['loser_break_points_return_total'][i] != 0:
        df['loser_any_break_points_chance?'][i] = 1

In [35]:
winner_break_point_convert_perc = []
loser_break_point_convert_perc = []

# Calculate % score
for i in range(0,len(df)):
    winner_break_point_convert_perc.append(df['winner_break_points_converted'][i]/df['winner_break_points_return_total'][i])
    loser_break_point_convert_perc.append(df['loser_break_points_converted'][i]/df['loser_break_points_return_total'][i])

# Create new columns with % score
df['winner_break_point_convert_%'] = winner_break_point_convert_perc
df['loser_break_point_convert_%'] = loser_break_point_convert_perc

In [36]:
for column in df.columns[-2:]:
    print(f"{column}: {df[column].isnull().sum()}")

winner_break_point_convert_%: 0
loser_break_point_convert_%: 10637


This is expected as it represents the opposite statistics from the break point save % as before, we will fill the null values with 0 as we did before.

In [37]:
df['loser_break_point_convert_%'].fillna(value = 0, inplace = True)

In [38]:
for column in df.columns[-2:]:
    print(f"{column}: {df[column].isnull().sum()}")

winner_break_point_convert_%: 0
loser_break_point_convert_%: 0


### Service Points Won %

__Service Points Won %__ is derived by <code>service_points_won</code> / <code>service_points_total</code>

In [39]:
winner_service_points_won_perc = []
loser_service_points_won_perc = []

# Calculate % score
for i in range(0, len(df)):
    winner_service_points_won_perc.append(df['winner_service_points_won'][i]/df['winner_service_points_total'][i])
    loser_service_points_won_perc.append(df['loser_service_points_won'][i]/df['loser_service_points_total'][i])

# Create new columns with % score
df['winner_service_points_won_%'] = winner_service_points_won_perc
df['loser_service_points_won_%'] = loser_service_points_won_perc

### Return Points Won %

__Return Points Won %__ is derived by <code>return_points_won</code> / <code>return_points_total</code>

In [40]:
winner_return_points_won_perc = []
loser_return_points_won_perc = []

# Calculate % score
for i in range(0, len(df)):
    winner_return_points_won_perc.append(df['winner_return_points_won'][i]/df['winner_return_points_total'][i])
    loser_return_points_won_perc.append(df['loser_return_points_won'][i]/df['loser_return_points_total'][i])

# Create new columns with % score
df['winner_return_points_won_%'] = winner_return_points_won_perc
df['loser_return_points_won_%'] = loser_return_points_won_perc

### Total Points Won %

__Total Points Won %__ is derived by <code>total_points_won</code> / <code>total_points_total</code>

In [41]:
winner_total_points_won_perc = []
loser_total_points_won_perc = []

# Calculate % score
for i in range(0, len(df)):
    winner_total_points_won_perc.append(df['winner_total_points_won'][i]/df['winner_total_points_total'][i])
    loser_total_points_won_perc.append(df['loser_total_points_won'][i]/df['loser_total_points_total'][i])

# Create new columns with % score
df['winner_total_points_won_%'] = winner_total_points_won_perc
df['loser_total_points_won_%'] = loser_total_points_won_perc

## Additional Metrics

### Aces:Double Faults Ratio

__Aces:Double Faults Ratio__ is derived by <code>aces</code> / <code>double_faults</code>

In [42]:
winner_aces_df_ratio = []
loser_aces_df_ratio = []

# Calculate % score
for i in range(0, len(df)):
    winner_aces_df_ratio.append(df['winner_aces'][i]/df['winner_double_faults'][i])
    loser_aces_df_ratio.append(df['loser_aces'][i]/df['loser_double_faults'][i])

# Create new columns with % score
df['winner_aces_df_ratio'] = winner_aces_df_ratio
df['loser_aces_df_ratio'] = loser_aces_df_ratio

In [43]:
for column in df.columns[-2:]:
    print(f"{column}: {df[column].isnull().sum()}")

winner_aces_df_ratio: 621
loser_aces_df_ratio: 1064


Expected result with all the players not hitting a single double fault during the match, again, we will convert them to 0.

In [44]:
df['winner_aces_df_ratio'].fillna(value = 0, inplace = True)
df['loser_aces_df_ratio'].fillna(value = 0, inplace = True)

In [45]:
for column in df.columns[-2:]:
    print(f"{column}: {df[column].isnull().sum()}")

winner_aces_df_ratio: 0
loser_aces_df_ratio: 0


### Aces Per Minute

__Aces Per Minute__ is derived by <code>aces</code> / <code>match_duration</code>

In [46]:
winner_aces_min = []
loser_aces_min = []

# Calculate % score
for i in range(0, len(df)):
    winner_aces_min.append(df['winner_aces'][i]/df['match_duration'][i])
    loser_aces_min.append(df['loser_aces'][i]/df['match_duration'][i])

# Create new columns with % score
df['winner_aces_per_min'] = winner_aces_min
df['loser_aces_per_min'] = loser_aces_min

### Difference In First Serves In %

__Difference In First Serves__ is derived by <code>winner_first_serves_in_%</code> - <code>loser_first_serves_in_%</code>

In [47]:
diff_in_first_serves_in_perc = []

# Calculate % score
for i in range(0, len(df)):
    diff_in_first_serves_in_perc.append(df['winner_first_serves_in_%'][i] - df['loser_first_serves_in_%'][i])

# Create new columns with % score
df['diff_in_first_serves_in_%'] = diff_in_first_serves_in_perc

### Serve Rating:Return Rating Ratio

__Serve Rating:Return Rating Ratio__ is derived by <code>serve_rating</code> / <code>return_rating</code>

In [48]:
winner_serve_return_rating_ratio = []
loser_serve_return_rating_ratio = []

# Calculate % score
for i in range(0, len(df)):
    winner_serve_return_rating_ratio.append(df['winner_serve_rating'][i]/df['winner_return_rating'][i])
    loser_serve_return_rating_ratio.append(df['loser_serve_rating'][i]/df['loser_return_rating'][i])

# Create new columns with % score
df['winner_serve_return_rating_ratio'] = winner_serve_return_rating_ratio
df['loser_serve_return_rating_ratio'] = loser_serve_return_rating_ratio

### Serve:Return Points Won Ratio

__Serve Rating:Return Points Won Ratio__ is derived by <code>service_points_won</code> / <code>return_points_won</code>

In [49]:
winner_serve_return_points_won_ratio = []
loser_serve_return_points_won_ratio = []

# Calculate % score
for i in range(0, len(df)):
    winner_serve_return_points_won_ratio.append(df['winner_service_points_won'][i]/df['winner_return_points_won'][i])
    loser_serve_return_points_won_ratio.append(df['loser_service_points_won'][i]/df['loser_return_points_won'][i])

# Create new columns with % score
df['winner_serve_return_points_won_ratio'] = winner_serve_return_points_won_ratio
df['loser_serve_return_points_won_ratio'] = loser_serve_return_points_won_ratio

### Difference in Total Return Points Played

__Difference in Total Return Points Played__ is derived by <code>return_points_total</code> - <code>return_points_total</code> (both ways)

In [50]:
winner_diff_in_return_points_played = []
loser_diff_in_return_points_played = []

# Calculate % score
for i in range(0, len(df)):
    winner_diff_in_return_points_played.append(df['winner_return_points_total'][i] - df['loser_return_points_total'][i])
    loser_diff_in_return_points_played.append(df['loser_return_points_total'][i] - df['winner_return_points_total'][i])
    
# Create new columns with % score
df['winner_diff_in_return_points_played'] = winner_diff_in_return_points_played
df['loser_diff_in_return_points_played'] = loser_diff_in_return_points_played

## Organizing the New Columns

In [51]:
# To view current column list
column_list = []
for column in df.columns:
    column_list.append(str(column))
column_list

['match_id',
 'year',
 'tourney_slug',
 'conditions',
 'surface',
 'match_duration',
 'winner_slug',
 'winner_serve_rating',
 'winner_aces',
 'winner_double_faults',
 'winner_first_serves_in',
 'winner_first_serves_total',
 'winner_first_serve_points_won',
 'winner_first_serve_points_total',
 'winner_second_serve_points_won',
 'winner_second_serve_points_total',
 'winner_break_points_saved',
 'winner_break_points_serve_total',
 'winner_service_games_played',
 'winner_return_rating',
 'winner_first_serve_return_won',
 'winner_first_serve_return_total',
 'winner_second_serve_return_won',
 'winner_second_serve_return_total',
 'winner_break_points_converted',
 'winner_break_points_return_total',
 'winner_return_games_played',
 'winner_service_points_won',
 'winner_service_points_total',
 'winner_return_points_won',
 'winner_return_points_total',
 'winner_total_points_won',
 'winner_total_points_total',
 'loser_slug',
 'loser_serve_rating',
 'loser_aces',
 'loser_double_faults',
 'loser_fir

In [52]:
# Restructuring column list
column_list = ['match_id', 'year', 'tourney_slug', 'conditions', 'surface', 'match_duration', 
# winner stats               
               'winner_slug', 'winner_serve_rating',  'winner_return_rating',  'winner_serve_return_rating_ratio',
               'winner_aces', 'winner_double_faults',  'winner_aces_df_ratio', 'winner_aces_per_min',
               'winner_first_serves_in', 'winner_first_serves_total', 'winner_first_serves_in_%',
               'winner_first_serve_points_won', 'winner_first_serve_points_total', 'winner_first_serves_points_won_%',
               'winner_second_serve_points_won', 'winner_second_serve_points_total', 'winner_second_serves_points_won_%',
               'winner_break_points_saved', 'winner_break_points_serve_total', 
               'winner_break_point_save_%',  'winner_faced_break_points?',
               'winner_service_games_played',  'winner_return_games_played', 
               'winner_first_serve_return_won', 'winner_first_serve_return_total',  'winner_first_serves_return_points_won_%',
               'winner_second_serve_return_won', 'winner_second_serve_return_total', 'winner_second_serves_return_points_won_%',
               'winner_break_points_converted', 'winner_break_points_return_total', 
               'winner_break_point_convert_%',  'winner_any_break_points_chance?',
               'winner_service_points_won', 'winner_service_points_total', 'winner_service_points_won_%',
               'winner_return_points_won', 'winner_return_points_total', 'winner_return_points_won_%',
               'winner_serve_return_points_won_ratio', 'winner_diff_in_return_points_played',
               'winner_total_points_won', 'winner_total_points_total', 'winner_total_points_won_%',
# loser stats
               'loser_slug', 'loser_serve_rating', 'loser_return_rating', 'loser_serve_return_rating_ratio',
               'loser_aces', 'loser_double_faults', 'loser_aces_df_ratio', 'loser_aces_per_min',
               'loser_first_serves_in', 'loser_first_serves_total', 'loser_first_serves_in_%',
               'loser_first_serve_points_won', 'loser_first_serve_points_total', 'loser_first_serves_points_won_%',
               'loser_second_serve_points_won', 'loser_second_serve_points_total', 'loser_second_serves_points_won_%',
               'loser_break_points_saved', 'loser_break_points_serve_total', 
               'loser_break_point_save_%', 'loser_faced_break_points?',
               'loser_service_games_played', 'loser_return_games_played',
               'loser_first_serve_return_won', 'loser_first_serve_return_total', 'loser_first_serves_return_points_won_%',
               'loser_second_serve_return_won', 'loser_second_serve_return_total', 'loser_second_serves_return_points_won_%',
               'loser_break_points_converted', 'loser_break_points_return_total', 
               'loser_break_point_convert_%', 'loser_any_break_points_chance?',
               'loser_service_points_won', 'loser_service_points_total','loser_service_points_won_%',
               'loser_return_points_won', 'loser_return_points_total', 'loser_return_points_won_%',
               'loser_serve_return_points_won_ratio', 'loser_diff_in_return_points_played',
               'loser_total_points_won', 'loser_total_points_total', 'loser_total_points_won_%',
# deltas
               'diff_in_first_serves_in_%'
              ]

In [53]:
df = df.reindex(columns = column_list)

In [54]:
# Cross-check for null values
for column in df.columns:
    print(f"{column}: {df[column].isnull().sum()}")

match_id: 0
year: 0
tourney_slug: 0
conditions: 0
surface: 0
match_duration: 0
winner_slug: 0
winner_serve_rating: 0
winner_return_rating: 0
winner_serve_return_rating_ratio: 0
winner_aces: 0
winner_double_faults: 0
winner_aces_df_ratio: 0
winner_aces_per_min: 0
winner_first_serves_in: 0
winner_first_serves_total: 0
winner_first_serves_in_%: 0
winner_first_serve_points_won: 0
winner_first_serve_points_total: 0
winner_first_serves_points_won_%: 0
winner_second_serve_points_won: 0
winner_second_serve_points_total: 0
winner_second_serves_points_won_%: 0
winner_break_points_saved: 0
winner_break_points_serve_total: 0
winner_break_point_save_%: 0
winner_faced_break_points?: 0
winner_service_games_played: 0
winner_return_games_played: 0
winner_first_serve_return_won: 0
winner_first_serve_return_total: 0
winner_first_serves_return_points_won_%: 0
winner_second_serve_return_won: 0
winner_second_serve_return_total: 0
winner_second_serves_return_points_won_%: 0
winner_break_points_converted: 0
w

## Short Summary of Section

In [55]:
print(f"In this section, we have added a total of 35 columns, describing various percentage scores and relevant metrics. The clean dataset now contains {df.shape[0]} rows x {df.shape[1]} columns.")

In this section, we have added a total of 35 columns, describing various percentage scores and relevant metrics. The clean dataset now contains 92159 rows x 95 columns.


In [56]:
# Save clean dataframe for easy accessibility for future use
df.to_csv('clean_df_v2.0.csv', index = False)

__The second stage of data cleaning and feature engineering is completed. We have successfully created a dataframe with 92159 rows x 95 columns. The dataframe was saved as a csv file named <code>clean_df_v2.0</code>__

# Data Cleaning (Individual Statistics Analysis)

This section aims to reformat the clean dataframe, <code>clean_df_v2.0</code> from a match-by-match basis to rows with individual statistical performance. Hence, the rows are expected to be doubled in this section, with an equal split between statistics that result in wins and losses. The final dataset produced by this section will be suitable for supervised learning model to understand significant individual statistics which improves the likelihood of winning a match.

In [64]:
df = pd.read_csv('clean_df_v2.0.csv')

In [58]:
# Identify winner and loser statistics
winner_stats = []
for column in df.columns:
    if str(column).startswith('winner_'):
        winner_stats.append(column)
        
loser_stats = []
for column in df.columns:
    if str(column).startswith('loser_'):
        loser_stats.append(column)

In [59]:
# Extracting only winning statistics
dfwin = df.drop(columns = loser_stats, axis = 1)

# Renaming winner_ into general name
for column in dfwin.columns:
    if str(column).startswith('winner_') == True:
        dfwin.rename(columns = {column: column[7:]}, inplace = True)

# Assigning boolean target variable "winner"
dfwin['winner'] = np.nan
dfwin['winner'].fillna(value = 1, inplace = True)

In [60]:
# Extracting only losing statistics
dfloss = df.drop(columns = winner_stats, axis = 1)

# Renaming winner_ into general name
for column in dfloss.columns:
    if str(column).startswith('loser_') == True:
        dfloss.rename(columns = {column: column[6:]}, inplace = True)

# Assigning boolean target variable "winner"
dfloss['winner'] = np.nan
dfloss['winner'].fillna(value = 0, inplace = True)

In [61]:
# Combine both win and loss dataframe
df = pd.concat([dfwin, dfloss], sort = False)

In [62]:
# Shuffle the dataframe, reset index and drop original index numbers
df = shuffle(df)
df.reset_index(inplace = True)
df.drop(columns = 'index', axis = 1, inplace = True)

In [63]:
df.to_csv('clean_df_v3.0.csv', index = False)

__Great ! We have achieved a reformatted dataset with each row representing individual performance of a given match and the match outcome ! The final clean dataset consists of 184318 rows × 52 columns. The dataset was saved as <code>clean_df_v3.0.csv</code>.__